# Import

In [1]:
import numpy as np
import pandas as pd
import os
import math

from scipy import stats #Analysis 
from scipy.stats import norm 

import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# # 한글 폰트가 깨지면 실행하세요! 
# # 한글 폰트가 따로 깔려있지 않은 컴퓨터에서는 해당 cell을 넘겨주세요!!! 
# import matplotlib.font_manager as fm
# fontlist = fm.findSystemFonts(fontpaths = None, fontext='ttf')
# # 아래 주석을 지워서 폰트 리스트를 확인하고 한글 폰트를 font_path에 추가합니다
# fontlist[:]

# from matplotlib import font_manager, rc
# #font_path = 'C:\\Users\\mtang\\AppData\\Local\\Microsoft\\Windows\\Fonts\\NanumSquare.ttf'
# font_path = 'C:\\WINDOWS\\Fonts\\NanumGothicLight.ttf'
# font = font_manager.FontProperties(fname=font_path).get_name()
# rc('font', family=font)

# 1. Data Load  

**치매 예방을 위한 라이프로그 치매 분류**

**9,327 rows × 66 columns**

For more details https://aihub.or.kr/problem_contest/nipa-learning-platform/6

In [3]:
path = "input/"

# original data : data 
data =  pd.read_csv(path + "demential_data.csv", parse_dates=['summary_date']) # 애초에 datatime 유형의 데이터를 읽어 올 때부터 형변환하기
data

,EMAIL,summary_date,activity_average_met,activity_cal_active,activity_cal_total,activity_class_5min,activity_daily_movement,activity_high,activity_inactive,activity_inactivity_alerts,...,sleep_temperature_deviation,sleep_temperature_trend_deviation,timezone,sleep_total,CONVERT(activity_class_5min USING utf8),CONVERT(activity_met_1min USING utf8),CONVERT(sleep_hr_5min USING utf8),CONVERT(sleep_hypnogram_5min USING utf8),CONVERT(sleep_rmssd_5min USING utf8),DIAG_NM
0,nia+404@rowan.kr,2020-11-27,1.71875,730,2944,...,14346,0,417,0,...,-0.12,99.99,NaN,\r,2/1/1/1/1/1/2/2/1/1/1/1/1/1/2/2/2/3/2/2/2/2/2/...,0.9/0.9/1.4/1.9/1.1/0.9/0.9/1.1/1.3/1/0.9/1.1/...,0/73/73/73/72/71/70/71/71/71/70/70/73/72/74/74...,4/2/4/3/3/1/2/2/2/2/2/2/3/3/3/4/4/3/2/2/2/2/2/...,0/10/10/10/11/11/10/12/18/13/14/12/10/10/18/17...,CN
1,nia+404@rowan.kr,2020-11-28,1.40625,342,2449,...,6352,0,473,0,...,-0.32,99.99,NaN,\r,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/2/2/2/2/2/...,1.2/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,69/70/69/69/70/72/71/72/70/69/69/69/68/68/63/6...,2/4/2/2/2/2/3/1/2/2/4/4/2/2/2/2/2/2/2/2/2/2/4/...,23/23/26/24/18/13/15/14/17/20/24/30/23/25/22/1...,CN
2,nia+404@rowan.kr,2020-11-29,1.46875,401,2544,...,7297,0,586,0,...,0.07,99.99,NaN,\r,1/1/1/1/1/1/1/2/1/1/1/1/2/2/2/2/2/1/1/1/1/1/2/...,1.1/1.1/1.2/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,0/74/73/73/74/74/74/71/71/70/70/69/70/68/66/69...,4/2/4/4/1/1/1/4/4/4/4/4/4/4/2/3/4/2/2/4/2/2/2/...,0/11/14/20/13/14/14/16/27/29/27/20/19/19/14/12...,CN
3,nia+404@rowan.kr,2020-11-30,0.34375,27,1850,...,491,0,176,0,...,-0.41,99.99,NaN,\r,2/1/2/2/1/2/1/1/2/1/1/1/1/1/2/1/1/1/1/1/2/2/2/...,0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/...,73/70/71/72/75/75/73/70/70/70/67/63/63/63/63/6...,4/4/4/4/3/3/3/2/4/4/4/2/2/2/2/2/2/2/2/4/2/2/2/...,24/28/19/17/12/10/17/20/23/23/25/31/26/25/34/3...,CN
4,nia+404@rowan.kr,2020-12-01,1.46875,333,2518,...,5861,0,646,0,...,-0.27,99.99,NaN,\r,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/2/3/3/2/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,0/0/0/0/0/0/0/0/69/69/71/69/65/66/64/64/65/66/...,4/4/4/4/4/4/4/4/4/4/4/2/2/2/2/3/3/2/4/4/4/2/2/...,0/0/0/0/0/0/0/0/21/22/26/23/19/29/22/17/14/13/...,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,nia+206@rowan.kr,2020-12-22,1.34375,227,2316,...,3863,3,735,2,...,-0.16,99.99,NaN,\r,2/1/1/2/2/1/1/1/1/2/2/3/2/1/1/1/1/1/1/1/1/1/1/...,1.1/1.4/1.2/0.9/1.2/1.1/0.9/0.9/0.9/1/0.9/0.9/...,0/54/54/54/55/56/0/0/55/52/52/53/54/54/56/57/6...,4/4/4/2/2/2/2/4/2/2/2/1/1/1/1/4/2/2/3/3/3/3/3/...,0/35/39/28/26/41/0/0/52/31/27/30/21/22/27/40/3...,CN
9323,nia+206@rowan.kr,2020-12-24,1.34375,249,2351,...,4411,1,780,4,...,-0.09,99.99,NaN,\r,1/1/1/1/1/1/1/1/1/1/2/3/2/2/2/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,68/66/67/67/68/69/69/70/71/71/71/69/72/70/70/7...,4/4/2/2/2/1/1/1/1/1/1/2/2/3/3/2/2/2/2/2/3/4/2/...,26/16/19/18/16/18/19/17/15/16/15/13/11/13/13/1...,CN
9324,nia+206@rowan.kr,2020-12-26,1.53125,570,2682,...,11057,1,518,1,...,0.41,99.99,NaN,\r,1/1/1/1/1/1/2/2/2/2/2/2/1/1/1/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/1.1/0.9/0.9/1.1/0.9/0.9/0.9/0....,0/0/65/65/66/67/69/72/73/72/73/74/73/74/75/75/...,4/4/2/2/2/2/2/2/2/2/2/2/4/4/4/4/2/3/3/3/2/2/4/...,0/0/13/12/12/13/12/12/12/16/13/12/13/17/13/18/...,CN
9325,nia+206@rowan.kr,2020-12-27,1.34375,295,2331,...,5135,4,578,0,...,0.27,99.99,NaN,\r,1/1/1/2/3/2/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/...,1.1/0.9/0.9/0.9/1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/...,0/0/0/64/64/64/64/65/65/66/66/66/66/68/68/68/6...,4/4/4/4/2/2/2/2/2/2/2/2/2/3/3/3/3/2/4/4/2/2/2/...,0/0/0/14/17/18/14/14/17/15/17/12/17/12/13/12/1...,CN


# 2. EDA  

**9,327 rows × 66 columns**

148명에 대한 데일리 라이프로그 데이터 
크게 걸음거리, 수면, 컨버팅 데이터로 나눌 수 있겠습니다. 

**basic (3 cols)**
- timezone 시간 장소 정보 
    - drop 
    - 완벽한 null variable 
- EMAIL
    - drop
    - 식별자 feature 
- summary_date

**activity (26 cols)**
- activity_class_5min, activity_met_1min, 하루간 5분당 활동 로그,하루간 1분당 MET 로그
    - drop 
    - BLOB type -> js로 읽을 수 있다고는 함 

**sleep (31 cols)**
- sleep_is_longest, 본 수면 여부 
    - 전부 다 1 
- sleep_temperature_trend_deviation, 피부 온도 경향 편차 
    - 전부 다 99.99
- sleep_total, 수면 시간 
    - 전부 다 \r
- sleep_temperature_deviation 
    - drop 
    - sleep_temperature_delta 랑 같은 변수 
    - sleep_temperature_deviation 을 drop 하고 delta를 남길 것 
- sleep_hr_5min, sleep_hypnogram_5min, sleep_rmssd_5min
    - 5분 당 심박동로그, 수면 상태 로그, 5분당 심박동변동 로그
    - drop 
    - BLOB type 

**converting**
- CONVERT(activity_class_5min USING utf8)    
- CONVERT(activity_met_1min USING utf8)      
- CONVERT(sleep_hr_5min USING utf8)          
- CONVERT(sleep_hypnogram_5min USING utf8)  
- CONVERT(sleep_rmssd_5min USING utf8) 

**y variable**
- DIAG_NM 
    - 정상(CN), 경도인지 장애(MCI), 치매(Dem)
    
So we may use 56 columns for training...

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9327 entries, 0 to 9326
Data columns (total 66 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   EMAIL                                     9327 non-null   object        
 1   summary_date                              9327 non-null   datetime64[ns]
 2   activity_average_met                      9327 non-null   float64       
 3   activity_cal_active                       9327 non-null   int64         
 4   activity_cal_total                        9327 non-null   int64         
 5   activity_class_5min                       9327 non-null   object        
 6   activity_daily_movement                   9327 non-null   int64         
 7   activity_high                             9327 non-null   int64         
 8   activity_inactive                         9327 non-null   int64         
 9   activity_inactivity_alerts    

# 3. Preprocessing 

**9,327 rows × 66 columns**

**9,327 rows × 51 columns**


**feature drop**
- timezone
- EMAIL
- sleep_temperature_trend_deviation, sleep_is_longest, sleep_total
    - 모든 값이 1, '\r', 99.99 이므로 제거 
- sleep_temperature_deviation
    - sleep_temperature_delta 변수와 중복되므로 deviation 변수를 제거
- activity_class_5min, activity_met_1min
    - convert data
- sleep_hr_5min, sleep_hypnogram_5min, sleep_rmssd_5min
    - convert data


- CONVERT(activity_class_5min USING utf8)    
- CONVERT(activity_met_1min USING utf8)      
- CONVERT(sleep_hr_5min USING utf8)          
- CONVERT(sleep_hypnogram_5min USING utf8)  
- CONVERT(sleep_rmssd_5min USING utf8) 

**y variable**
- DIAG_NM 
    - 정상(CN), 경도인지 장애(MCI), 치매(Dem)
    
So we may use 56 columns for training...

In [5]:
# column의 수가 많기 때문에 효율적으로 관리하기 위해서 
# column의 이름들을 묶어서 진행합시다
drop_cols = ['timezone', 'EMAIL',
             'activity_class_5min', 'activity_met_1min', 
             'sleep_hr_5min', 'sleep_hypnogram_5min', 'sleep_rmssd_5min',
             'sleep_is_longest', 'sleep_temperature_trend_deviation', 'sleep_total', 'sleep_temperature_deviation']
convert_cols = data.iloc[:, 60:65].columns

In [6]:
# data -> processed_data 
# Drop Features 
processed_data = data.drop(drop_cols, axis=1).copy()
processed_data = processed_data.drop(convert_cols, axis=1).copy()
processed_data

,summary_date,activity_average_met,activity_cal_active,activity_cal_total,activity_daily_movement,activity_high,activity_inactive,activity_inactivity_alerts,activity_low,activity_medium,...,sleep_score,sleep_score_alignment,sleep_score_deep,sleep_score_disturbances,sleep_score_efficiency,sleep_score_latency,sleep_score_rem,sleep_score_total,sleep_temperature_delta,DIAG_NM
0,2020-11-27,1.71875,730,2944,14346,0,417,0,545,47,...,62,100,41,50,27,97,66,62,-0.12,CN
1,2020-11-28,1.40625,342,2449,6352,0,473,0,392,8,...,54,100,49,43,46,91,35,44,-0.32,CN
2,2020-11-29,1.46875,401,2544,7297,0,586,0,362,24,...,61,100,56,47,34,89,41,62,0.07,CN
3,2020-11-30,0.34375,27,1850,491,0,176,0,34,1,...,55,100,35,40,44,86,29,56,-0.41,CN
4,2020-12-01,1.46875,333,2518,5861,0,646,0,377,6,...,56,100,64,55,39,14,54,59,-0.27,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,2020-12-22,1.34375,227,2316,3863,3,735,2,155,17,...,76,100,53,46,74,97,84,82,-0.16,CN
9323,2020-12-24,1.34375,249,2351,4411,1,780,4,167,22,...,77,100,34,57,76,81,95,86,-0.09,CN
9324,2020-12-26,1.53125,570,2682,11057,1,518,1,332,61,...,65,100,1,56,69,86,89,64,0.41,CN
9325,2020-12-27,1.34375,295,2331,5135,4,578,0,173,30,...,81,100,9,59,86,89,98,97,0.27,CN


In [7]:
processed_data = processed_data.drop('summary_date', axis=1).copy()
processed_data

,activity_average_met,activity_cal_active,activity_cal_total,activity_daily_movement,activity_high,activity_inactive,activity_inactivity_alerts,activity_low,activity_medium,activity_met_min_high,...,sleep_score,sleep_score_alignment,sleep_score_deep,sleep_score_disturbances,sleep_score_efficiency,sleep_score_latency,sleep_score_rem,sleep_score_total,sleep_temperature_delta,DIAG_NM
0,1.71875,730,2944,14346,0,417,0,545,47,0,...,62,100,41,50,27,97,66,62,-0.12,CN
1,1.40625,342,2449,6352,0,473,0,392,8,0,...,54,100,49,43,46,91,35,44,-0.32,CN
2,1.46875,401,2544,7297,0,586,0,362,24,0,...,61,100,56,47,34,89,41,62,0.07,CN
3,0.34375,27,1850,491,0,176,0,34,1,0,...,55,100,35,40,44,86,29,56,-0.41,CN
4,1.46875,333,2518,5861,0,646,0,377,6,0,...,56,100,64,55,39,14,54,59,-0.27,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,1.34375,227,2316,3863,3,735,2,155,17,21,...,76,100,53,46,74,97,84,82,-0.16,CN
9323,1.34375,249,2351,4411,1,780,4,167,22,7,...,77,100,34,57,76,81,95,86,-0.09,CN
9324,1.53125,570,2682,11057,1,518,1,332,61,7,...,65,100,1,56,69,86,89,64,0.41,CN
9325,1.34375,295,2331,5135,4,578,0,173,30,28,...,81,100,9,59,86,89,98,97,0.27,CN


# 4. Data Split 

모든 전처리와 y variable labeling이 완료되었다. train / valid / test data로 분할하고 용도에 맞게 model을 돌리도록 하자. 

In [8]:
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, GridSearchCV

# processed_data -> input_data 
input_data = processed_data.copy()

# y 변수와 X 변수 분할 관리 
input_data_y = input_data['DIAG_NM'].copy()
input_data_X = input_data.drop(['DIAG_NM'], axis=1)

In [9]:
# LabelEncoder
# y data를 LabelEncdoer로 한 번 더 labeling 합니다 -> 100만원, 10000만원... = 0, 1, ... 
# 추후에 DL에서 output을 맞춰주기 위함입니다, 최종 마지막에서 원래 label값(100만원, 1000만원...)으로 되돌립니다
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
input_data_y = le.fit_transform(list(input_data_y))
print("original gain_label \n", le.classes_)
print("gain_lable label \n", set(input_data_y))

original gain_label 
 ['CN' 'Dem' 'MCI']
gain_lable label 
 {0, 1, 2}


In [11]:
# train/ test data 로 split 
tr_val_X, test_X, tr_val_y, test_y = train_test_split(
    input_data_X, 
    input_data_y, 
    test_size = 0.2, 
    random_state = 42,
    shuffle=True,
    stratify = input_data_y # Classification 중요 option
)

# RNN 모델 제작 

In [12]:
#!pip install tensorflow

In [13]:
import math
import sklearn
import sklearn.preprocessing
import datetime
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# split data in 80%/10%/10% train/validation/test sets
valid_set_size_percentage = 10 
test_set_size_percentage = 10 

#display parent directory and working directory
print(os.path.dirname(os.getcwd())+':', os.listdir(os.path.dirname(os.getcwd())));
print(os.getcwd()+':', os.listdir(os.getcwd()));

Instructions for updating:
non-resource variables are not supported in the long term
C:\Users\bhs89\machine-learning: ['.ipynb_checkpoints', 'cache', 'datasets_for_clustering', 'dataset_for_clustering.ipynb', 'Machine_learning', 'Team_project']
C:\Users\bhs89\machine-learning\Team_project: ['-p', '.ipynb_checkpoints', 'baseline_hong-Copy1.ipynb', 'baseline_hong.ipynb', 'Dementia_df_hong.ipynb', 'EDA_report_code_dementia.ipynb', 'input', 'RNN', 'RNN.ipynb', 'saved_model']


In [14]:
cols_convert = data.columns[60:66]
data_convert = data[cols_convert].copy()
data_convert

,CONVERT(activity_class_5min USING utf8),CONVERT(activity_met_1min USING utf8),CONVERT(sleep_hr_5min USING utf8),CONVERT(sleep_hypnogram_5min USING utf8),CONVERT(sleep_rmssd_5min USING utf8),DIAG_NM
0,2/1/1/1/1/1/2/2/1/1/1/1/1/1/2/2/2/3/2/2/2/2/2/...,0.9/0.9/1.4/1.9/1.1/0.9/0.9/1.1/1.3/1/0.9/1.1/...,0/73/73/73/72/71/70/71/71/71/70/70/73/72/74/74...,4/2/4/3/3/1/2/2/2/2/2/2/3/3/3/4/4/3/2/2/2/2/2/...,0/10/10/10/11/11/10/12/18/13/14/12/10/10/18/17...,CN
1,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/2/2/2/2/2/...,1.2/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,69/70/69/69/70/72/71/72/70/69/69/69/68/68/63/6...,2/4/2/2/2/2/3/1/2/2/4/4/2/2/2/2/2/2/2/2/2/2/4/...,23/23/26/24/18/13/15/14/17/20/24/30/23/25/22/1...,CN
2,1/1/1/1/1/1/1/2/1/1/1/1/2/2/2/2/2/1/1/1/1/1/2/...,1.1/1.1/1.2/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,0/74/73/73/74/74/74/71/71/70/70/69/70/68/66/69...,4/2/4/4/1/1/1/4/4/4/4/4/4/4/2/3/4/2/2/4/2/2/2/...,0/11/14/20/13/14/14/16/27/29/27/20/19/19/14/12...,CN
3,2/1/2/2/1/2/1/1/2/1/1/1/1/1/2/1/1/1/1/1/2/2/2/...,0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/...,73/70/71/72/75/75/73/70/70/70/67/63/63/63/63/6...,4/4/4/4/3/3/3/2/4/4/4/2/2/2/2/2/2/2/2/4/2/2/2/...,24/28/19/17/12/10/17/20/23/23/25/31/26/25/34/3...,CN
4,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/2/3/3/2/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,0/0/0/0/0/0/0/0/69/69/71/69/65/66/64/64/65/66/...,4/4/4/4/4/4/4/4/4/4/4/2/2/2/2/3/3/2/4/4/4/2/2/...,0/0/0/0/0/0/0/0/21/22/26/23/19/29/22/17/14/13/...,CN
...,...,...,...,...,...,...
9322,2/1/1/2/2/1/1/1/1/2/2/3/2/1/1/1/1/1/1/1/1/1/1/...,1.1/1.4/1.2/0.9/1.2/1.1/0.9/0.9/0.9/1/0.9/0.9/...,0/54/54/54/55/56/0/0/55/52/52/53/54/54/56/57/6...,4/4/4/2/2/2/2/4/2/2/2/1/1/1/1/4/2/2/3/3/3/3/3/...,0/35/39/28/26/41/0/0/52/31/27/30/21/22/27/40/3...,CN
9323,1/1/1/1/1/1/1/1/1/1/2/3/2/2/2/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,68/66/67/67/68/69/69/70/71/71/71/69/72/70/70/7...,4/4/2/2/2/1/1/1/1/1/1/2/2/3/3/2/2/2/2/2/3/4/2/...,26/16/19/18/16/18/19/17/15/16/15/13/11/13/13/1...,CN
9324,1/1/1/1/1/1/2/2/2/2/2/2/1/1/1/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/1.1/0.9/0.9/1.1/0.9/0.9/0.9/0....,0/0/65/65/66/67/69/72/73/72/73/74/73/74/75/75/...,4/4/2/2/2/2/2/2/2/2/2/2/4/4/4/4/2/3/3/3/2/2/4/...,0/0/13/12/12/13/12/12/12/16/13/12/13/17/13/18/...,CN
9325,1/1/1/2/3/2/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/...,1.1/0.9/0.9/0.9/1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/...,0/0/0/64/64/64/64/65/65/66/66/66/66/68/68/68/6...,4/4/4/4/2/2/2/2/2/2/2/2/2/3/3/3/3/2/4/4/2/2/2/...,0/0/0/14/17/18/14/14/17/15/17/12/17/12/13/12/1...,CN


#### - CONVERT(activity_class_5min USING utf8)

In [15]:
signal_list = list(data_convert['CONVERT(activity_class_5min USING utf8)'].str.split('/'))
signal_activity_class_5min = pd.DataFrame(signal_list).drop(288, axis=1).astype('float')
y_df = data_convert['DIAG_NM'] 
signal_activity_class_5min = pd.concat([signal_activity_class_5min, y_df], axis=1)
signal_activity_class_5min

,0,1,2,3,4,5,6,7,8,9,...,279,280,281,282,283,284,285,286,287,DIAG_NM
0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,...,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
3,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
9323,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
9324,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
9325,1.0,1.0,1.0,2.0,3.0,2.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN


#### - CONVERT(activity_met_1min USING utf8)

In [16]:
signal_list = list(data_convert['CONVERT(activity_met_1min USING utf8)'].str.split('/'))
signal_activity_met_1min = pd.DataFrame(signal_list).drop(1440, axis=1).astype('float')
signal_activity_met_1min = pd.concat([signal_activity_met_1min, y_df], axis=1)
signal_activity_met_1min

,0,1,2,3,4,5,6,7,8,9,...,1431,1432,1433,1434,1435,1436,1437,1438,1439,DIAG_NM
0,0.9,0.9,1.4,1.9,1.1,0.9,0.9,1.1,1.3,1.0,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
1,1.2,1.1,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
2,1.1,1.1,1.2,1.1,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
4,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,1.1,1.4,1.2,0.9,1.2,1.1,0.9,0.9,0.9,1.0,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
9323,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
9324,0.9,0.9,0.9,0.9,1.1,0.9,0.9,1.1,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
9325,1.1,0.9,0.9,0.9,1.0,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN


#### - CONVERT(sleep_hr_5min USING utf8)

In [17]:
signal_list = list(data_convert['CONVERT(sleep_hr_5min USING utf8)'].str.split('/'))
signal_sleep_hr_5min = pd.DataFrame(signal_list)
signal_sleep_hr_5min

,0,1,2,3,4,5,6,7,8,9,...,172,173,174,175,176,177,178,179,180,181
0,0,73,73,73,72,71,70,71,71,71,...,None,None,None,None,None,None,None,None,None,None
1,69,70,69,69,70,72,71,72,70,69,...,None,None,None,None,None,None,None,None,None,None
2,0,74,73,73,74,74,74,71,71,70,...,None,None,None,None,None,None,None,None,None,None
3,73,70,71,72,75,75,73,70,70,70,...,None,None,None,None,None,None,None,None,None,None
4,0,0,0,0,0,0,0,0,69,69,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,0,54,54,54,55,56,0,0,55,52,...,None,None,None,None,None,None,None,None,None,None
9323,68,66,67,67,68,69,69,70,71,71,...,None,None,None,None,None,None,None,None,None,None
9324,0,0,65,65,66,67,69,72,73,72,...,None,None,None,None,None,None,None,None,None,None
9325,0,0,0,64,64,64,64,65,65,66,...,None,None,None,None,None,None,None,None,None,None


In [18]:
signal_sleep_hr_5min = signal_sleep_hr_5min.drop(181, axis=1).transpose()
# 결측치 처리 Nan, '', ' ' -> 우선 다 int type 0으로 replace 
signal_sleep_hr_5min = signal_sleep_hr_5min.fillna(0) 
signal_sleep_hr_5min = signal_sleep_hr_5min.replace('', 0)
signal_sleep_hr_5min = signal_sleep_hr_5min.replace(' ', 0)
signal_sleep_hr_5min = signal_sleep_hr_5min.astype('float')

In [19]:
signal_sleep_hr_5min  = signal_sleep_hr_5min.transpose()
signal_sleep_hr_5min = pd.concat([signal_sleep_hr_5min, y_df], axis=1)
signal_sleep_hr_5min

,0,1,2,3,4,5,6,7,8,9,...,172,173,174,175,176,177,178,179,180,DIAG_NM
0,0.0,73.0,73.0,73.0,72.0,71.0,70.0,71.0,71.0,71.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
1,69.0,70.0,69.0,69.0,70.0,72.0,71.0,72.0,70.0,69.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
2,0.0,74.0,73.0,73.0,74.0,74.0,74.0,71.0,71.0,70.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
3,73.0,70.0,71.0,72.0,75.0,75.0,73.0,70.0,70.0,70.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,69.0,69.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,0.0,54.0,54.0,54.0,55.0,56.0,0.0,0.0,55.0,52.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9323,68.0,66.0,67.0,67.0,68.0,69.0,69.0,70.0,71.0,71.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9324,0.0,0.0,65.0,65.0,66.0,67.0,69.0,72.0,73.0,72.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9325,0.0,0.0,0.0,64.0,64.0,64.0,64.0,65.0,65.0,66.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN


#### - CONVERT(sleep_hypnogram_5min USING utf8)

In [20]:
signal_list = list(data_convert['CONVERT(sleep_hypnogram_5min USING utf8)'].str.split('/'))
signal_sleep_hypnogram_5min = pd.DataFrame(signal_list)

signal_sleep_hypnogram_5min = signal_sleep_hypnogram_5min.drop(180, axis=1).transpose()
# 결측치 처리 Nan, '', ' ' -> 우선 다 int type 0으로 replace 
signal_sleep_hypnogram_5min = signal_sleep_hypnogram_5min.fillna(0) 
signal_sleep_hypnogram_5min = signal_sleep_hypnogram_5min.replace('', 0)
signal_sleep_hypnogram_5min = signal_sleep_hypnogram_5min.replace(' ', 0)
signal_sleep_hypnogram_5min = signal_sleep_hypnogram_5min.astype('float')

In [21]:
signal_sleep_hypnogram_5min  = signal_sleep_hypnogram_5min.transpose()
signal_sleep_hypnogram_5min = pd.concat([signal_sleep_hypnogram_5min, y_df], axis=1)
signal_sleep_hypnogram_5min

,0,1,2,3,4,5,6,7,8,9,...,171,172,173,174,175,176,177,178,179,DIAG_NM
0,4.0,2.0,4.0,3.0,3.0,1.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
1,2.0,4.0,2.0,2.0,2.0,2.0,3.0,1.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
2,4.0,2.0,4.0,4.0,1.0,1.0,1.0,4.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
3,4.0,4.0,4.0,4.0,3.0,3.0,3.0,2.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
4,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,4.0,4.0,4.0,2.0,2.0,2.0,2.0,4.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9323,4.0,4.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9324,4.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9325,4.0,4.0,4.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN


#### - CONVERT(sleep_rmssd_5min USING utf8)

In [22]:
signal_list = list(data_convert['CONVERT(sleep_rmssd_5min USING utf8)'].str.split('/'))
signal_sleep_rmssd_5min = pd.DataFrame(signal_list)

signal_sleep_rmssd_5min = signal_sleep_rmssd_5min.drop(181, axis=1).transpose()
# 결측치 처리 Nan, '', ' ' -> 우선 다 int type 0으로 replace 
signal_sleep_rmssd_5min = signal_sleep_rmssd_5min.fillna(0) 
signal_sleep_rmssd_5min = signal_sleep_rmssd_5min.replace('', 0)
signal_sleep_rmssd_5min = signal_sleep_rmssd_5min.replace(' ', 0)
signal_sleep_rmssd_5min = signal_sleep_rmssd_5min.astype('float')

In [23]:
signal_sleep_rmssd_5min  = signal_sleep_rmssd_5min.transpose()
signal_sleep_rmssd_5min = pd.concat([signal_sleep_rmssd_5min, y_df], axis=1)
signal_sleep_rmssd_5min

,0,1,2,3,4,5,6,7,8,9,...,172,173,174,175,176,177,178,179,180,DIAG_NM
0,0.0,10.0,10.0,10.0,11.0,11.0,10.0,12.0,18.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
1,23.0,23.0,26.0,24.0,18.0,13.0,15.0,14.0,17.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
2,0.0,11.0,14.0,20.0,13.0,14.0,14.0,16.0,27.0,29.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
3,24.0,28.0,19.0,17.0,12.0,10.0,17.0,20.0,23.0,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,0.0,35.0,39.0,28.0,26.0,41.0,0.0,0.0,52.0,31.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9323,26.0,16.0,19.0,18.0,16.0,18.0,19.0,17.0,15.0,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9324,0.0,0.0,13.0,12.0,12.0,13.0,12.0,12.0,12.0,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9325,0.0,0.0,0.0,14.0,17.0,18.0,14.0,14.0,17.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN


In [24]:
signal1 = signal_activity_class_5min.values.tolist()
signal2 = signal_activity_met_1min.values.tolist()
signal3 = signal_sleep_hr_5min.values.tolist()
signal4 = signal_sleep_hypnogram_5min.values.tolist()
signal5 = signal_sleep_rmssd_5min.values.tolist()

# Data Split

In [25]:
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, GridSearchCV

# y 변수와 X 변수 분할 관리 
input_data_y = y_df.copy()
input_data_X1 = signal_activity_class_5min.drop(labels = 'DIAG_NM', axis = 1)
input_data_X2 = signal_activity_met_1min.drop(labels = 'DIAG_NM', axis = 1)
input_data_X3 = signal_sleep_hr_5min.drop(labels = 'DIAG_NM', axis = 1)
input_data_X4 = signal_sleep_hypnogram_5min.drop(labels = 'DIAG_NM', axis = 1)
input_data_X5 = signal_sleep_rmssd_5min.drop(labels = 'DIAG_NM', axis = 1)

In [26]:
input_data_y

0       CN
1       CN
2       CN
3       CN
4       CN
        ..
9322    CN
9323    CN
9324    CN
9325    CN
9326    CN
Name: DIAG_NM, Length: 9327, dtype: object

In [27]:
# OneHotEncoder

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)
ohe

OneHotEncoder(sparse=False)

In [28]:
input_data_y.shape
input_data_y.values.reshape(-1,1).shape

(9327, 1)

In [29]:
ohe.fit(input_data_y.values.reshape(-1,1))
one_hot_encoded = ohe.transform(input_data_y.values.reshape(-1,1))
one_hot_encoded

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [30]:
print(type(ohe.categories_))
ohe.categories_

<class 'list'>


[array(['CN', 'Dem', 'MCI'], dtype=object)]

In [31]:
ohe_df = pd.DataFrame(one_hot_encoded, columns=ohe.categories_[0])
ohe_df

,CN,Dem,MCI
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0
...,...,...,...
9322,1.0,0.0,0.0
9323,1.0,0.0,0.0
9324,1.0,0.0,0.0
9325,1.0,0.0,0.0


In [32]:
input_data_y  = ohe_df.copy()
# # input_data_y = ohe.categories_.copy()
# input_data_y = one_hot_encoded.copy()
input_data_y

,CN,Dem,MCI
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0
...,...,...,...
9322,1.0,0.0,0.0
9323,1.0,0.0,0.0
9324,1.0,0.0,0.0
9325,1.0,0.0,0.0


# Data Split

In [33]:
input_data_X1 = signal_activity_class_5min.drop(labels = 'DIAG_NM', axis = 1)
input_data_X2 = signal_activity_met_1min.drop(labels = 'DIAG_NM', axis = 1)
input_data_X3 = signal_sleep_hr_5min.drop(labels = 'DIAG_NM', axis = 1)
input_data_X4 = signal_sleep_hypnogram_5min.drop(labels = 'DIAG_NM', axis = 1)
input_data_X5 = signal_sleep_rmssd_5min.drop(labels = 'DIAG_NM', axis = 1)

In [34]:
type(input_data_X1)

pandas.core.frame.DataFrame

**y variable**
- DIAG_NM 
    - 정상(CN), 치매(Dem), 경도인지 장애(MCI)

In [35]:
input_data_X1.shape

(9327, 288)

In [36]:
input_data_y.shape

(9327, 3)

In [37]:
X1_train, X1_test, y1_train, y1_test = train_test_split(input_data_X1, input_data_y, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=2022)

In [38]:
X1_train, X1_val, y1_train, y1_val = train_test_split(X1_train, y1_train, test_size=0.2, shuffle=True, stratify=y1_train, random_state=2022)

In [39]:
print('훈련용  activity_class_5min: {}'.format(len(X1_train)))
print('검증용  activity_class_5min: {}'.format(len(X1_val)))
print('테스트용 activity_class_5min : {}'.format(len(X1_test)))
# num_classes = len(set(y1_train))
# print('label: {}'.format(num_classes))

훈련용  activity_class_5min: 5968
검증용  activity_class_5min: 1493
테스트용 activity_class_5min : 1866


In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Activation, SimpleRNN
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from keras.layers import Bidirectional
from keras import optimizers
from sklearn.metrics import accuracy_score
from keras.layers import Bidirectional
from keras.wrappers.scikit_learn import KerasClassifier

import tensorflow as tf

In [41]:
print("X1_train :" ,X1_train.shape)
print("X1_val :" ,X1_val.shape)
print("y1_train :" ,y1_train.shape)

X1_train : (5968, 288)
X1_val : (1493, 288)
y1_train : (5968, 3)


In [42]:
X1_train.shape

(5968, 288)

In [43]:
type(X1_train)

pandas.core.frame.DataFrame

In [44]:
type(y1_train)

pandas.core.frame.DataFrame

In [45]:
X1_train.shape

(5968, 288)

In [46]:
X2_train, X2_test, y2_train, y2_test = train_test_split(input_data_X2, input_data_y, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=2022)
X3_train, X3_test, y3_train, y3_test = train_test_split(input_data_X3, input_data_y, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=2022)
X4_train, X4_test, y4_train, y4_test = train_test_split(input_data_X4, input_data_y, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=2022)
X5_train, X5_test, y5_train, y5_test = train_test_split(input_data_X5, input_data_y, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=2022)

In [47]:
print('훈련용  activity_class_5min: {}'.format(len(X1_train)))
print('테스트용 activity_class_5min : {}'.format(len(X1_test)))
num_classes = len(set(y1_train))
print('label: {}'.format(num_classes))

훈련용  activity_class_5min: 5968
테스트용 activity_class_5min : 1866
label: 3


In [48]:
print("X3 :" ,X3_train.shape)
print("y3 :" ,y3_train.shape)
print("X4 :" ,X4_train.shape)
print("y4 :" ,y4_train.shape)
print("X5 :" ,X5_train.shape)
print("y5 :" ,y5_train.shape)

X3 : (7461, 181)
y3 : (7461, 3)
X4 : (7461, 180)
y4 : (7461, 3)
X5 : (7461, 181)
y5 : (7461, 3)


In [49]:
X3_train.shape[0]

7461

In [50]:
X3_train

,0,1,2,3,4,5,6,7,8,9,...,171,172,173,174,175,176,177,178,179,180
7985,0.0,70.0,74.0,77.0,79.0,71.0,71.0,71.0,81.0,77.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
632,0.0,0.0,74.0,75.0,74.0,74.0,74.0,77.0,76.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6863,0.0,0.0,0.0,54.0,53.0,53.0,54.0,54.0,55.0,55.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5359,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7136,57.0,58.0,58.0,59.0,57.0,56.0,56.0,61.0,57.0,56.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,0.0,0.0,56.0,56.0,55.0,54.0,54.0,54.0,55.0,56.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8477,0.0,0.0,0.0,0.0,53.0,0.0,53.0,55.0,54.0,54.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6660,0.0,0.0,0.0,0.0,67.0,67.0,69.0,69.0,69.0,69.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1805,0.0,87.0,87.0,86.0,88.0,86.0,86.0,91.0,88.0,89.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
X3_train =  X3_train.to_numpy()
X3_train

array([[ 0., 70., 74., ...,  0.,  0.,  0.],
       [ 0.,  0., 74., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0., 87., 87., ...,  0.,  0.,  0.],
       [64., 60., 59., ...,  0.,  0.,  0.]])

In [52]:
X3_test =  X3_test.to_numpy()
X3_test

array([[ 0., 55., 55., ...,  0.,  0.,  0.],
       [ 0.,  0., 66., ...,  0.,  0.,  0.],
       [ 0., 66., 66., ...,  0.,  0.,  0.],
       ...,
       [ 0., 78., 74., ...,  0.,  0.,  0.],
       [ 0., 50., 50., ...,  0.,  0.,  0.],
       [62., 62., 63., ...,  0.,  0.,  0.]])

In [53]:
X3_train = X3_train.reshape(X3_train.shape[0],X3_train.shape[1],1)
#X3_val = X3_val.reshape(X3_val.shape[0],X3_val.shape[1],1)
X3_test = X3_test.reshape(X3_test.shape[0],X3_test.shape[1],1)

In [54]:
def vanilla_rnn():
    model = Sequential()
    model.add(SimpleRNN(3, input_shape = (181,1), return_sequences = False)) #input shape에 열 개수 넣어주면 됨
    model.add(Dense(3)) #target 개수 = 3
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss='categorical_crossentropy', optimizer = adam, metrics = ['accuracy']) #label encoding 했기때문에  'sparse_categorical_crossentropy' 사용
    
    return model

In [55]:
# def vanilla_rnn():
#     model = Sequential()
#     #input shape에 열 개수 넣어주면 됨
#     model.add(SimpleRNN(20, input_shape = (288,1), return_sequences = False)) 
#     #model.add(SimpleRNN(20, return_sequences = False)) 
#     #target 개수 = 3
#     model.add(Dense(3)) 
#     model.add(Activation('softmax'))
    
#     adam = optimizers.Adam(lr = 0.001)
#     #one-hot encoding 했기때문에  'sparse_categorical_crossentropy' 사용
#     model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy']) 
    
#     return model

In [56]:
model = KerasClassifier(build_fn = vanilla_rnn, epochs = 10, batch_size = 256, verbose = 1)
model.fit(X3_train, y3_train)
y3_pred = model.predict(X3_test)


Train on 7461 samples
Epoch 1/10
7461/7461 [==============================] - 0s 52us/sample - loss: 1.0919 - acc: 0.2955
Epoch 2/10
7461/7461 [==============================] - 0s 42us/sample - loss: 1.0668 - acc: 0.2943
Epoch 3/10
7461/7461 [==============================] - 0s 42us/sample - loss: 1.0354 - acc: 0.3319
Epoch 4/10
7461/7461 [==============================] - 0s 42us/sample - loss: 1.0049 - acc: 0.3329
Epoch 5/10
7461/7461 [==============================] - 0s 42us/sample - loss: 0.9709 - acc: 0.4143
Epoch 6/10
7461/7461 [==============================] - 0s 41us/sample - loss: 0.9348 - acc: 0.6254
Epoch 7/10
7461/7461 [==============================] - 0s 43us/sample - loss: 0.9013 - acc: 0.6392
Epoch 8/10
7461/7461 [==============================] - 0s 42us/sample - loss: 0.8742 - acc: 0.6384
Epoch 9/10
7461/7461 [==============================] - 0s 42us/sample - loss: 0.8547 - acc: 0.6388
Epoch 10/10
7461/7461 [==============================] - 0s 41us/sample - loss

In [57]:
len(y3_pred)

1866

In [58]:
y3_pred

array([0, 0, 0, ..., 0, 0, 0])

In [60]:
label = input_data['DIAG_NM'].copy()
# LabelEncoder
# y data를 LabelEncdoer로 한 번 더 labeling 합니다 -> 100만원, 10000만원... = 0, 1, ... 
# 추후에 DL에서 output을 맞춰주기 위함입니다, 최종 마지막에서 원래 label값(100만원, 1000만원...)으로 되돌립니다
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
label_y = le.fit_transform(list(label))
print("original gain_label \n", le.classes_)
print("gain_lable label \n", set(label_y))

original gain_label 
 ['CN' 'Dem' 'MCI']
gain_lable label 
 {0, 1, 2}


In [62]:
y3_test

,CN,Dem,MCI
4405,0.0,0.0,1.0
4719,0.0,1.0,0.0
2772,1.0,0.0,0.0
8190,1.0,0.0,0.0
738,0.0,1.0,0.0
...,...,...,...
8008,0.0,0.0,1.0
2045,1.0,0.0,0.0
2174,0.0,0.0,1.0
1473,0.0,0.0,1.0


In [61]:
len(label_y)

9327

In [ ]:
print("accouracy : ", accuracy_score(y3_pred, y3_test))

#### CONVERT(activity_met_1min USING utf8) 에 대해 RNN 모델로 예측해봄.

148명에 대한 데일리 라이프로그 데이터 크게 걸음거리, 수면, 컨버팅 데이터로 나눌 수 있겠습니다.  
- 내가 원하는거는 DIAN_NM에 따른 activity_met_1min 패턴 예측
- 이로써 얻을 수 있는 것 : 정상과 비교하여 주로 환자들이 띄는 경향성 파악
- 좀 더 디테일한 비교를 원한다면 여러 convert데이터에 대해 동시 비교진행

In [ ]:
signal_df2_group = signal_df2.groupby('DIAG_NM')
#signal_df2_group = signal_df2_group.mean()
signal_df2_group = signal_df2_group.mean().transpose()
signal_df2_group

# # 3개 끊어서 평균 찍어보려고 나눠봤습니다 
# # 앞에서부터 3 토막 나눴습니다
# #signal_df2['mean_1'] = signal_df2.iloc[:, :3109].mean(axis=1)
# #signal_df2['mean_2'] = signal_df2.iloc[:, 3109:6218].mean(axis=1)
# #signal_df2['mean_3'] = signal_df2.iloc[:, 6218:9327].mean(axis=1)

# signal_df2_group.head()

**y variable**
- DIAG_NM 
    - 정상(CN), 치매(Dem), 경도인지 장애(MCI)

In [ ]:
plt.style.use("ggplot")
plt.figure(figsize=(15,10))
plt.suptitle("activity_met_1min", fontsize=20)

cols = signal_df2_group.columns
for i in range(3):
    plt.plot(signal_df2_group[cols[i]])
    plt.plot(signal_df2_group[cols[i]], label = str(cols[i]))
plt.legend()
plt.show()

In [ ]:
# function to create train, validation, test data given stock data and sequence length
def load_data(stock, seq_len):
    data_raw = stock.to_numpy() # convert to numpy array
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - seq_len): 
        data.append(data_raw[index: index + seq_len])
    
    data = np.array(data);
    valid_set_size = int(np.round(valid_set_size_percentage/100*data.shape[0]));  
    test_set_size = int(np.round(test_set_size_percentage/100*data.shape[0]));
    train_set_size = data.shape[0] - (valid_set_size + test_set_size);
    
    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]
    
    x_valid = data[train_set_size:train_set_size+valid_set_size,:-1,:]
    y_valid = data[train_set_size:train_set_size+valid_set_size,-1,:]
    
    x_test = data[train_set_size+valid_set_size:,:-1,:]
    y_test = data[train_set_size+valid_set_size:,-1,:]
    
    return [x_train, y_train, x_valid, y_valid, x_test, y_test]

CONVERT(activity_met_1min USING utf8) 에 대해 RNN 모델로 예측해봄.
- signal_df2_group

In [ ]:
activity_met_1min_CN = pd.DataFrame(signal_df2_group["CN"])
activity_met_1min_CN = activity_met_1min_CN.transpose()
activity_met_1min_CN

In [ ]:
# create train, test data
seq_len = 20 # choose sequence length
x_train, y_train, x_valid, y_valid, x_test, y_test = load_data(activity_met_1min_CN, seq_len)
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ', y_train.shape)
print('x_valid.shape = ',x_valid.shape)
print('y_valid.shape = ', y_valid.shape)
print('x_test.shape = ', x_test.shape)
print('y_test.shape = ',y_test.shape)

In [ ]:
# # function to get the next batch
# def get_next_batch(batch_size):
#     global index_in_epoch, x_train, perm_array   
#     start = index_in_epoch
#     index_in_epoch += batch_size

#     if index_in_epoch > x_train.shape[0]:
#         np.random.shuffle(perm_array) # shuffle permutation array
#         start = 0 # start next epoch
#         index_in_epoch = batch_size

#     end = index_in_epoch
#     return x_train[perm_array[start:end]], y_train[perm_array[start:end]]

In [ ]:
# def RNN_model(n_steps, 
#     n_inputs = 3, 
#     n_neurons = 200, 
#     n_outputs = 3,
#     n_layers = 2,
#     learning_rate = 0.001,
#     batch_size = 1,
#     n_epochs = 100,
#     train_set_size = x_train.shape[0],
#     test_set_size = x_test.shape[0]):
#     seq_len = 20 # choose sequence length
#     x_train, y_train, x_valid, y_valid, x_test, y_test = load_data(signal_df2_group, seq_len)
#     print('x_train.shape = ',x_train.shape)
#     print('y_train.shape = ', y_train.shape)
#     print('x_valid.shape = ',x_valid.shape)
#     print('y_valid.shape = ', y_valid.shape)
#     print('x_test.shape = ', x_test.shape)
#     print('y_test.shape = ',y_test.shape)
    
#     # from tensorflow.python.framework import ops
#     # ops.reset_default_graph()
#     # import tensorflow.compat.v1 as tf
#     # tf_.disable_v2_behavior()
#     tf.set_random_seed(2022)

#     ## Basic Cell RNN in tensorflow

#     index_in_epoch = 0;
#     perm_array  = np.arange(x_train.shape[0])
#     np.random.shuffle(perm_array)
    
#     #tf.reset_default_graph()
#     X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
#     y = tf.placeholder(tf.float32, [None, n_outputs])
    
#     # use Basic RNN Cell
#     # layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.elu)
#     #           for layer in range(n_layers)]
#     layers = [tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons, activation=tf.nn.elu)
#               for layer in range(n_layers)]
#     # layers =tf.keras.layers.SimpleRNNCell(units=n_neurons) #n_neurons= 출력의 크기

#     # use Basic LSTM Cell 
#     # layers = [tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons, activation=tf.nn.elu)
#     #          for layer in range(n_layers)]

#     # use LSTM Cell with peephole connections
#     # layers = [tf.nn.rnn_cell.LSTMCell(num_units=n_neurons, 
#     #                                  activation=tf.nn.leaky_relu, use_peepholes = True)
#     #          for layer in range(n_layers)]

#     # use GRU cell
#     #layers = [tf.contrib.rnn.GRUCell(num_units=n_neurons, activation=tf.nn.leaky_relu)
#     #          for layer in range(n_layers)]

#     multi_layer_cell = tf.keras.layers.StackedRNNCells(layers)
#     rnn_outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

#     stacked_rnn_outputs = tf.reshape(rnn_outputs, [-1, n_neurons]) 
#     stacked_outputs = tf.layers.dense(stacked_rnn_outputs, n_outputs)
#     outputs = tf.reshape(stacked_outputs, [-1, n_steps, n_outputs])
#     outputs = outputs[:,n_steps-1,:] # keep only last output of sequence

#     loss = tf.reduce_mean(tf.square(outputs - y)) # loss function = mean squared error 
#     optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate) 
#     training_op = optimizer.minimize(loss)
    
#     # run graph
#     with tf.Session() as sess: 
#         sess.run(tf.global_variables_initializer())
#         for iteration in range(int(n_epochs*train_set_size/batch_size)):
#             x_batch, y_batch = get_next_batch(batch_size) # fetch the next training batch 
#             sess.run(training_op, feed_dict={X: x_batch, y: y_batch}) 
#             if iteration % int(5*train_set_size/batch_size) == 0:
#                 mse_train = loss.eval(feed_dict={X: x_train, y: y_train}) 
#                 mse_valid = loss.eval(feed_dict={X: x_valid, y: y_valid}) 
#                 print('%.2f epochs: MSE train/valid = %.6f/%.6f'%(
#                     iteration*batch_size/train_set_size, mse_train, mse_valid))

#         y_train_pred = sess.run(outputs, feed_dict={X: x_train})
#         y_valid_pred = sess.run(outputs, feed_dict={X: x_valid})
#         y_test_pred = sess.run(outputs, feed_dict={X: x_test})


In [ ]:
# tf.set_random_seed(2022)

# ## Basic Cell RNN in tensorflow

# index_in_epoch = 0;
# perm_array  = np.arange(x_train.shape[0])
# np.random.shuffle(perm_array)

# RNN_model = RNN_model(n_steps = seq_len -1, 
#     n_inputs = 3, 
#     n_neurons = 200, 
#     n_outputs = 3,
#     n_layers = 2,
#     learning_rate = 0.001,
#     batch_size = 1,
#     n_epochs = 100,
#     train_set_size = x_train.shape[0],
#     test_set_size = x_test.shape[0])

In [ ]:
# from tensorflow.python.framework import ops
# ops.reset_default_graph()
# import tensorflow.compat.v1 as tf
# tf_.disable_v2_behavior()

#tf.random.set_seed(2022)
tf.set_random_seed(2022)

## Basic Cell RNN in tensorflow

index_in_epoch = 0;
perm_array  = np.arange(x_train.shape[0])
np.random.shuffle(perm_array)

# function to get the next batch
def get_next_batch(batch_size):
    global index_in_epoch, x_train, perm_array   
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > x_train.shape[0]:
        np.random.shuffle(perm_array) # shuffle permutation array
        start = 0 # start next epoch
        index_in_epoch = batch_size
        
    end = index_in_epoch
    return x_train[perm_array[start:end]], y_train[perm_array[start:end]]

# parameters
n_steps = seq_len-1 
n_inputs = 3 
n_neurons = 200 
n_outputs = 3
n_layers = 2
learning_rate = 0.001
batch_size = 1
n_epochs = 100 
train_set_size = x_train.shape[0]
test_set_size = x_test.shape[0]

In [ ]:
#tf.reset_default_graph()

# X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_outputs])

In [ ]:
# use Basic RNN Cell
# layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.elu)
#           for layer in range(n_layers)]
layers = [tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons, activation=tf.nn.elu)
          for layer in range(n_layers)]
# layers =tf.keras.layers.SimpleRNNCell(units=n_neurons) #n_neurons= 출력의 크기

# use Basic LSTM Cell 
# layers = [tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons, activation=tf.nn.elu)
#          for layer in range(n_layers)]

# use LSTM Cell with peephole connections
# layers = [tf.nn.rnn_cell.LSTMCell(num_units=n_neurons, 
#                                  activation=tf.nn.leaky_relu, use_peepholes = True)
#          for layer in range(n_layers)]

# use GRU cell
#layers = [tf.contrib.rnn.GRUCell(num_units=n_neurons, activation=tf.nn.leaky_relu)
#          for layer in range(n_layers)]
                                                                     
multi_layer_cell = tf.nn.rnn_cell.MultiRNNCell(layers)

In [ ]:
rnn_outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

In [ ]:
stacked_rnn_outputs = tf.reshape(rnn_outputs, [-1, n_neurons]) 
stacked_outputs = tf.layers.dense(stacked_rnn_outputs, n_outputs)
outputs = tf.reshape(stacked_outputs, [-1, n_steps, n_outputs])
outputs = outputs[:,n_steps-1,:] # keep only last output of sequence
                                              
loss = tf.reduce_mean(tf.square(outputs - y)) # loss function = mean squared error 
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate) 
training_op = optimizer.minimize(loss)

In [ ]:
# run graph
with tf.Session() as sess: 
    sess.run(tf.global_variables_initializer())
    for iteration in range(int(n_epochs*train_set_size/batch_size)):
        x_batch, y_batch = get_next_batch(batch_size) # fetch the next training batch 
        sess.run(training_op, feed_dict={X: x_batch, y: y_batch}) 
        if iteration % int(5*train_set_size/batch_size) == 0:
            mse_train = loss.eval(feed_dict={X: x_train, y: y_train}) 
            mse_valid = loss.eval(feed_dict={X: x_valid, y: y_valid}) 
            print('%.2f epochs: MSE train/valid = %.6f/%.6f'%(
                iteration*batch_size/train_set_size, mse_train, mse_valid))

    y_train_pred = sess.run(outputs, feed_dict={X: x_train})
    y_valid_pred = sess.run(outputs, feed_dict={X: x_valid})
    y_test_pred = sess.run(outputs, feed_dict={X: x_test})

In [ ]:
y_train.shape

In [ ]:
ft = 0 # 0 = open, 1 = close, 2 = highest, 3 = lowest

## show predictions
plt.figure(figsize=(15, 5));
plt.subplot(1,2,1);

plt.plot(np.arange(y_train.shape[0]), y_train[:,ft], color='blue', label='train target')

plt.plot(np.arange(y_train.shape[0], y_train.shape[0]+y_valid.shape[0]), y_valid[:,ft],
         color='gray', label='valid target')

plt.plot(np.arange(y_train.shape[0]+y_valid.shape[0],
                   y_train.shape[0]+y_test.shape[0]+y_test.shape[0]),
         y_test[:,ft], color='black', label='test target')

plt.plot(np.arange(y_train_pred.shape[0]),y_train_pred[:,ft], color='red',
         label='train prediction')

plt.plot(np.arange(y_train_pred.shape[0], y_train_pred.shape[0]+y_valid_pred.shape[0]),
         y_valid_pred[:,ft], color='orange', label='valid prediction')

plt.plot(np.arange(y_train_pred.shape[0]+y_valid_pred.shape[0],
                   y_train_pred.shape[0]+y_valid_pred.shape[0]+y_test_pred.shape[0]),
         y_test_pred[:,ft], color='green', label='test prediction')

plt.title('Existing data activity_met_1min USING utf8')
plt.xlabel('activity_met_1min USING utf8')
plt.ylabel('acitvity_amout')
plt.legend(loc='best');

plt.subplot(1,2,2);

plt.plot(np.arange(y_train.shape[0], y_train.shape[0]+y_test.shape[0]),
         y_test[:,ft], color='black', label='test target')

plt.plot(np.arange(y_train_pred.shape[0], y_train_pred.shape[0]+y_test_pred.shape[0]),
         y_test_pred[:,ft], color='green', label='test prediction')

plt.title('Prediction of activity_met_1min USING utf8')
plt.xlabel('activity_met_1min USING utf8')
plt.ylabel('acitvity_amout')
plt.legend(loc='best');

corr_price_development_train = np.sum(np.equal(np.sign(y_train[:,1]-y_train[:,0]),
            np.sign(y_train_pred[:,1]-y_train_pred[:,0])).astype(int)) / y_train.shape[0]
corr_price_development_valid = np.sum(np.equal(np.sign(y_valid[:,1]-y_valid[:,0]),
            np.sign(y_valid_pred[:,1]-y_valid_pred[:,0])).astype(int)) / y_valid.shape[0]
corr_price_development_test = np.sum(np.equal(np.sign(y_test[:,1]-y_test[:,0]),
            np.sign(y_test_pred[:,1]-y_test_pred[:,0])).astype(int)) / y_test.shape[0]

print('correct sign prediction for close - open price for train/valid/test: %.2f/%.2f/%.2f'%(
    corr_price_development_train, corr_price_development_valid, corr_price_development_test))

In [ ]:
tf.reset_default_graph()